# Lecture 22: Multiple Linear Regression
***

We'll need Numpy, Matplotlib, Pandas, and scipy.stats for this notebook, so let's load them. 

In [ ]:
import numpy as np 
from scipy import stats
import pandas as pd
import matplotlib.pylab as plt 
%matplotlib inline

### Exercise 1 - Advertising Budgets and Simple Linear Regression
*** 

The data in advertising.csv concerns the sales of a particular product in 200 different markets, along with advertising budgets for each market for three different media types: TV, Radio, and Newspaper.  The sales feature is given in thousands of units, and each of the advertising budget features are given in thousands of dollars.


**Part A**: Load the data into a Pandas DataFrame. 

In [ ]:
dfAd = pd.read_csv("data/advertising.csv")
dfAd.head()

**Part B**: Run the following code to make three separate scatter plots depicting the relationships between each of the features, $\texttt{TV}$, $\texttt{radio}$, and $\texttt{news}$ on the response variable $\texttt{sales}$. From the plots, can you determine whether there appears to be a relationship between each of the advertising media types and the sales of the product? 

In [ ]:
media = ["tv", "radio", "news"]

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(8,16))
axes[0].scatter(dfAd["tv"], dfAd["sales"])

for axi, m in enumerate(media): 
    axes[axi].scatter(dfAd[m], dfAd["sales"], color="steelblue")
    axes[axi].grid(alpha=0.25)
    axes[axi].set_xlabel(m, fontsize=16)
    axes[axi].set_ylabel("sales", fontsize=16)

**Part C**: Use [stats.linregress](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.linregress.html) to fit three simple linear regression models to the data of the form 

$$
Y_i = \alpha + \beta x_i + \epsilon_i, \quad \textrm{for } i=1,2, \ldots, n
$$

where $x$ is a particular media type and  $\epsilon \sim N(0,\sigma^2)$. Interpret your results.   

In [ ]:
media = ["tv", "radio", "news"]

for m in media: 
    print("SLR for {} vs sales".format(m))
    print("----------------------")
    bhat, ahat, rval, pval, stderr = # TODO 
    print("intercept = {:.4f}".format(ahat))
    print("slope = {:.4f}".format(bhat))
    print("p-value = {}".format(pval))
    print("\n")

### Exercise 2 - Advertising Budgets and Multiple Linear Regression
*** 

In this exercise you'll fit a multiple linear regression model to the advertising data.  For this we'll need a package that we haven't encountered yet called [statsmodels](http://www.statsmodels.org/stable/index.html).  We load it as follows.  Note that you might get a FutureWarning from Pandas.  This is expected. 


In [ ]:
import statsmodels.api as sm 

**Part A**: The following code will fit an MLR model to the data. Note that the add_constant function is necessary in order to include an intercept term in the MLR model. 

In [ ]:
# Collect the features in a 2D array 
X = dfAd[["tv", "radio", "news"]]

# Add a constant to the array for the intecept 
X = sm.add_constant(X)

# Collect the response data in an array 
y = dfAd["sales"]

# Fit the ordinary least-squares (OLS) model 
model = sm.OLS(y, X).fit()

The estimated parameters are for the model are stored in model.params.  In this case, because we got $X$ from a Pandas DataFrame, the resulting model parameters are stored in a Pandas Series. 

In [ ]:
print(model.params)

**Part C**: Based on the parameters estimated by the model, replace the unknown $\hat{\beta}_j$ parameters below with the actual values in the model 

$$
\texttt{sales} = \hat{\beta}_0 + \hat{\beta}_1 \times \texttt{TV} + \hat{\beta}_2 \times \texttt{radio} + \hat{\beta}_3 \times \texttt{news}
$$

**Part D**: Compare and contrast the model parameters obtained from multiple linear regression with those obtained from the individual simple linear regression models obtained in Exercise 1.  Does something seem fishy? 

**Part E**: We can compute the pair-wise correlation between each of the features directly from the DataFrame using Pandas .corr() function. 

In [ ]:
dfAd[["tv", "radio", "news"]].corr()

Does any pair of features seem particularly correlated?  Can you use this to explain the fishiness observed in **Part D**? 

### Exercise 3 - Polynomial Regression via Multiple Linear Regression 
*** 

It's not too difficult to believe that some relationships between features and the response are nonlinear. Consider the following example, where the single feature $x$ and the response $y$ have a quadratic relationship of the form 

$$
Y = \frac{1}{4} - X + X^2 + \epsilon 
$$

The following code samples $n=30$ data points from the true model, fits the SLR model, and plots it against the data. It should be clear from both the picture and the $R^2$-value that the SLR model is a very poor fit of the data. 

In [ ]:
# Generate Data
n, sig = 30, 0.05
x = np.linspace(0,1,n) + 0.05*np.random.rand(n) 
y = 0.25 - x + x**2 + stats.norm.rvs(0,sig,size=n)

# Fit SLR model 
bhat, ahat, rval, pval, stderr = stats.linregress(x, y)

# Plot data and SLR model 
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
xplot = np.linspace(np.min(x), np.max(x))
ax.plot(xplot, ahat + bhat*xplot, color="steelblue", lw=3)
ax.scatter(x, y, color="steelblue", s=100, alpha=0.9)
ax.grid(alpha=0.25)
ax.set_xlabel("X", fontsize=16)
ax.set_ylabel("Y", fontsize=16)
ax.text(0.8, np.min(y)+.00, r"$R^2$={:.3f}".format(rval**2), fontsize=20);

**Part A**: We can fit a **polynomial** model to the single-feature data be thinking of the polynomial features as features in a multiple linear regression model.  If we make the association  

$$
x_1 = x \quad \textrm{and} \quad x_2 = x^2 
$$

then we can fit a multiple linear regression of the form 

$$
\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x_1 + \hat{\beta}_2 x_2 
$$

Let's see how we can do this in python. 

In [ ]:
# Collect the linear feature x and it's square in an array 
X = np.column_stack((x, x**2))

# Add a constant to the array for the intecept 
X = sm.add_constant(X)

# Fit the ordinary least-squares (OLS) model 
polymodel = sm.OLS(y, X).fit()

# Print estimated parameters 
print(polymodel.params)

**Part B**: Write down the estimated MLR model in terms of the features $x_1$ and $x_2$ as well as the interpretation of the associated polynomial model in terms of the single feature $x$. Does this model seem close to the true model that the data was generated from?  

**Part C**: Modify the code below to plot the obtained polynomial regression model against the data.  How does it look?  

In [ ]:
# Plot data and SLR model 
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
xplot = np.linspace(np.min(x), np.max(x))
yplot = #TODO 
ax.plot(xplot, yplot, color="steelblue", lw=3)
ax.scatter(x, y, color="steelblue", s=100, alpha=0.9)
ax.grid(alpha=0.25)
ax.set_xlabel("X", fontsize=16)
ax.set_ylabel("Y", fontsize=16);

### Exercise 4 - Guessing Polynomial Features from Residual Plots 
*** 

In the previous exercise we looked for a polynomial regression model that was quadratic because we already knew the true form of the model (which never happens in practice).  If we don't know how many polynomial features to include in our regression model, we can gain insight into which features might be missing by plotting the associated residuals.  Recall that the residual is the difference between the true response value $y$ and the response predicted by the model, $\hat{y}$: 

$$
r_i = y_i - \hat{y}_i 
$$

**Part A**: Suppose that your data really is linear.  What do you expect a scatter plot of the residuals to look like? 

**Note**: Before continuing, scroll down and execute the cell in the **Helper Functions** section at the bottom of this notebook. 

**Part B**: The following code samples data from a true linear model, fits a SLR model to it, and plots the model and the associated residuals.  Does the residual plot agree with what you concluded in **Part A**? 

In [ ]:
X, y = nonlinear_data(dataset=0)
fit_and_res_plot(X, y)

**Part C**: Now consider a data set that is not linear.  What do you notice about the associated residual plot? 

In [ ]:
X, y = nonlinear_data(dataset=1)
fit_and_res_plot(X, y)

**Part C**: What polynomial feature do you think is missing?  Complete the following code be adding the missing feature and running the fit again.  What does the residual plot look like now? 

In [ ]:
X, y = nonlinear_data(dataset=1)
X = np.column_stack((X, # TODO 
fit_and_res_plot(X, y)

**Part C**: OK, one more dataset.  See if you can figure out the missing features and obtain a polynomial regression model that fits the data. 

In [ ]:
X, y = nonlinear_data(dataset=2)
fit_and_res_plot(X, y)

<br><br><br><br>
<br><br><br><br>
<br><br><br><br>
<br><br><br><br>

### Helper Functions
***

In [ ]:
def nonlinear_data(dataset, n=30):
    
    if dataset < 0 or dataset > 2: 
        print("Datasets must be numbered 0-2.  Defaulting to 0.")
        dataset = 0
    
    np.random.seed(1237)
    
    X = np.linspace(0,1,n) + .05 * np.random.rand(n)
    
    if dataset == 0: 
       return  X, 0.5 + 0.75 * X + .5*np.random.rand(n)
    elif dataset == 1: 
       return  X, 0.25 - X + X*X + .1*np.random.rand(n)
    elif dataset == 2: 
       return  X, 2*(3*(2*x-1.2)**3 + 2*(2*x-1.2)**2 -(2*x-1.2)) + 1.5*np.random.rand(n)

def fit_and_res_plot(X, y, hint=False):
    
    if len(X.shape)==1:
        x = X
    else:
        x = X[:,0]
    
    # Fit Data 
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit() 
    
    # Print fitted model 
    print("Fitted PLR Model: yhat = ", end="")
    for p, beta in enumerate(model.params):
        print("{:.3f}".format(beta), end="")
        if p > 0: 
            print(" x^{} ".format(p), end="")
        else:
            print(" ", end="")
        if p != len(model.params)-1:
            print("+ ", end="")
            
    # Plot data/fit 
    yhat = model.predict(X)
    res = y - yhat 
    xp = np.linspace(np.min(x), np.max(x), 50)
    yp = np.zeros(len(xp))
    for p, beta in enumerate(model.params):
        yp += beta * xp**p 
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
    axes[0].scatter(x, y, color="steelblue", s=100, alpha=0.9)
    axes[0].plot(xp, yp, color="steelblue", lw=3)
    axes[0].grid(alpha=0.25)
    axes[0].set_xlabel("X", fontsize=16)
    axes[0].set_ylabel("Y", fontsize=16)
    axes[0].set_title("Data and PLR Fit", fontsize=16)
    
    # Plot residuals 
    axes[1].plot([np.min(x), np.max(x)], [0, 0], lw=2, ls="--", alpha=0.5, color="black")
    axes[1].scatter(x, res, color="#6a9373", s=100, alpha=0.9)
    axes[1].grid(alpha=0.25)
    axes[1].set_xlabel("X", fontsize=16)
    axes[1].set_ylabel("Res", fontsize=16)
    axes[1].set_title("Residual Plot", fontsize=16)
    